## **PREDICTING THE IDEAL LOCATION FOR OPENNING A SPA CENTRE IN SINGAPORE**

### **Background and Problem**

### Recently, I have met a friend who wishes to open a spa to provide facial, massage,  and body treatment services in Singapore. However, He is struggling with the location for opening his first outlet.  Since I enrolled in IBM Data Science Course, I hope I could use what I have learned from this course to help my friend to solve this problem.

### Singapore is a sovereign island city-state in Southeast Asia., and is a highly developed country and is ranked 9th on the UN Human Development Index, the highest in Asia for a sovereign state, with the 7th highest GDP per capita in the world. My friends' new spa outlet is going to targetting the mid-age population group with good spending power. She has a limited rental budget. The places located near the Shopping center with convenient public transport are ideal to choose from. Also, she hopes there is less competition near the place. So which places are ideal for considerations? Let us explore!

### **Data Description**

### First of all, I obtained the Singapore 2020 Q1 retail rental dataset from https://www.ura.gov.sg/-/media/Corporate/Property/Commercial-Properties-2020Q1/comm_median_rentals_2020Q1.pdf?la=en, the original file is a pdf file. I tried to using Tabula.read_pdf to extract the table from the pdf file but failed to do so in the end. Thus, I downloaded the data directly from the link and converted it to CSV file in my computer. 
### Secondly, I uploaded to IBM Db2 on cloud. Use SQL magic retrieve the data and convert to dataframe.
### Thirdly, I used geopy to get all the streets' "Lat" and "long" in the dataframe.
### Lastly, I used Foursquare API to find out and analysis the nearby venues for all the streets in the dataframe.

In [7]:
%load_ext sql

In [8]:
%sql ibm_db_sa://dcg13641:7wds%5Eqtlbbpvkl5m@dashdb-txn-sbox-yp-dal09-08.services.dal.bluemix.net:50000/BLUDB

'Connected: dcg13641@BLUDB'

In [4]:
data = %sql select * from Retail

 * ibm_db_sa://dcg13641:***@dashdb-txn-sbox-yp-dal09-08.services.dal.bluemix.net:50000/BLUDB
Done.


In [9]:
data = %sql select * from Retail
df = data.DataFrame()
df.head()

 * ibm_db_sa://dcg13641:***@dashdb-txn-sbox-yp-dal09-08.services.dal.bluemix.net:50000/BLUDB
Done.


,Street,Median_Rent____psm__per_month_
0,AIRPORT BOULEVARD,270.63
1,ALEXANDRA ROAD,102.91
2,ANG MO KIO AVENUE 3,303.79
3,ANSON ROAD,75.36
4,BALESTIER ROAD,76.67


In [11]:
pip install geopy

     |████████████████████████████████| 112kB 5.1MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [12]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [13]:
df['Street'] = df['Street'] + ', sg'
df.head()

,Street,Median_Rent____psm__per_month_
0,"AIRPORT BOULEVARD, sg",270.63
1,"ALEXANDRA ROAD, sg",102.91
2,"ANG MO KIO AVENUE 3, sg",303.79
3,"ANSON ROAD, sg",75.36
4,"BALESTIER ROAD, sg",76.67


In [14]:
geolocator = Nominatim()
df['gcode'] = df['Street'].apply(geolocator.geocode)
df.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  """Entry point for launching an IPython kernel.


,Street,Median_Rent____psm__per_month_,gcode
0,"AIRPORT BOULEVARD, sg",270.63,"(Airport Boulevard, Changi, Southeast, 207663,..."
1,"ALEXANDRA ROAD, sg",102.91,"(Alexandra Road, Bukit Merah, Singapore, Centr..."
2,"ANG MO KIO AVENUE 3, sg",303.79,"(Ang Mo Kio Avenue 3, Ang Mo Kio, Singapore, C..."
3,"ANSON ROAD, sg",75.36,"(Anson Road, Radin Mas, Bukit Merah, Singapore..."
4,"BALESTIER ROAD, sg",76.67,"(Balestier Road, Novena, Singapore, Central, 3..."


In [15]:
df['lat'] = [g.latitude for g in df.gcode]
df['long'] = [g.longitude for g in df.gcode]
df.head()

,Street,Median_Rent____psm__per_month_,gcode,lat,long
0,"AIRPORT BOULEVARD, sg",270.63,"(Airport Boulevard, Changi, Southeast, 207663,...",1.358623,103.989130
1,"ALEXANDRA ROAD, sg",102.91,"(Alexandra Road, Bukit Merah, Singapore, Centr...",1.283903,103.801557
2,"ANG MO KIO AVENUE 3, sg",303.79,"(Ang Mo Kio Avenue 3, Ang Mo Kio, Singapore, C...",1.369781,103.838831
3,"ANSON ROAD, sg",75.36,"(Anson Road, Radin Mas, Bukit Merah, Singapore...",1.271805,103.843321
4,"BALESTIER ROAD, sg",76.67,"(Balestier Road, Novena, Singapore, Central, 3...",1.322346,103.852741


### Define Foursquare Credentials and Version

In [17]:
CLIENT_ID = 'FPURLTN0HP4EJMMJQLQRBPK1UGEBPPFFKENYJSCUEPFSKCR5' # your Foursquare ID
CLIENT_SECRET = 'AH4B4BY31VLEI2ZF3G3YNGSFUL3UDQ1KZ5I2JXAJ3JUJRMDM' # your Foursquare Secret
VERSION = '20180604'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FPURLTN0HP4EJMMJQLQRBPK1UGEBPPFFKENYJSCUEPFSKCR5
CLIENT_SECRET:AH4B4BY31VLEI2ZF3G3YNGSFUL3UDQ1KZ5I2JXAJ3JUJRMDM


In [28]:
neighborhood_latitude = df.loc[0, 'lat']
neighborhood_longitude = df.loc[0, 'long'] 

neighborhood_name = df.loc[0, 'Street'] 

In [29]:
LIMIT= 100
radius=500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=FPURLTN0HP4EJMMJQLQRBPK1UGEBPPFFKENYJSCUEPFSKCR5&client_secret=AH4B4BY31VLEI2ZF3G3YNGSFUL3UDQ1KZ5I2JXAJ3JUJRMDM&v=20180604&ll=1.3586234,103.9891295&radius=500&limit=100'

In [30]:
results = requests.get(url).json()

In [31]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [32]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues)


filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]


nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)


nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Jewel Changi Airport,Shopping Mall,1.360119,103.989790
1,HSBC Rain Vortex,Waterfall,1.360151,103.989740
2,Birds Of Paradise,Ice Cream Shop,1.360363,103.989088
3,Crowne Plaza Changi Airport,Hotel,1.358561,103.987967
4,Tim Ho Wan,Dim Sum Restaurant,1.360219,103.989151


### Let's convert Dalian to its latitude and longitude coordinates.

In [41]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [42]:
Street_venues = getNearbyVenues(names=df['Street'],
                                   latitudes=df['lat'],
                                   longitudes=df['long']
                                  )

AIRPORT BOULEVARD, sg
ALEXANDRA ROAD, sg
ANG MO KIO AVENUE 3, sg
ANSON ROAD, sg
BALESTIER ROAD, sg
BAYFRONT AVENUE, sg
BEACH ROAD, sg
BENCOOLEN STREET, sg
BUKIT TIMAH ROAD, sg
CHANGI BUSINESS PARK CENTRAL 1, sg
CHANGI ROAD, sg
CHOA CHU KANG AVENUE 4, sg
CHOA CHU KANG NORTH 6, sg
CLEMENCEAU AVENUE, sg
COLEMAN STREET, sg
COLLYER QUAY, sg
COMMONWEALTH AVENUE WEST, sg
CUPPAGE ROAD, sg
CUSCADEN ROAD, sg
EAST COAST ROAD, sg
EU TONG SEN STREET, sg
GEYLANG ROAD, sg
GOPENG STREET, sg
GUILLEMARD ROAD, sg
HAVELOCK ROAD, sg
HIGH STREET, sg
HOLLAND AVENUE, sg
HOUGANG AVENUE 10, sg
HOUGANG STREET 51, sg
IRRAWADDY ROAD, sg
JALAN ANAK BUKIT, sg
JALAN BERSEH, sg
JALAN BESAR, sg
JALAN JURONG KECHIL, sg
JALAN MASJID, sg
JELEBU ROAD, sg
JURONG EAST STREET 21, sg
JURONG GATEWAY ROAD, sg
JURONG WEST CENTRAL 2, sg
JURONG WEST CENTRAL 3, sg
KITCHENER ROAD, sg
KOEK ROAD, sg
LORONG KILAT, sg
MACPHERSON ROAD, sg
MARINA BOULEVARD, sg
MARINE PARADE ROAD, sg
MARITIME SQUARE, sg
MIDDLE ROAD, sg
MOUNTBATTEN ROAD, sg


In [45]:
Street_onehot = pd.get_dummies(Street_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Street_onehot['Neighborhood'] = Street_venues['Neighborhood']

# move neighborhood column to the first column
fixed_columns = [Street_onehot.columns[-1]] + list(Street_onehot.columns[:-1])
Street_onehot = Street_onehot[fixed_columns]

Street_onehot.head()

,Yoga Studio,ATM,Accessories Store,Airport Lounge,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Stadium,Basketball Court,Bay,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bistro,Boarding House,Boat or Ferry,Bookstore,Border Crossing,Boutique,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buddhist Temple,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Campground,Canal,Candy Store,Cantonese Restaurant,Casino,Cha Chaan Teng,Cheese Shop,Chinese Breakfast Place,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Theater,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store,English Restaurant,Event Space,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hainan Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hong Kong Restaurant,Hookah Bar,Hospital,Hostel,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Indoor Play Area,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Laundromat,Leather Goods Store,Light Rail Station,Lighthouse,Lighting Store,Lingerie Store,Lounge,Luggage Store,Macanese Restaurant,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Mosque,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Night Market,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoor Sculpture,Outlet Store,Paella Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Peking Duck Restaurant,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Pier,Pizza Place,Playground,Plaza,Pool,Pop-Up Shop,Portuguese Restaurant,Pub,Public Art,Racetrack,Ramen Restaurant,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,River,Rock Climbing Spot,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Satay Restaurant,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Area,Smoke Shop,Snack Place,Soba Restaurant,Soccer Field,Soccer Stadium,Soup Place,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stables,Stadium,Stationery Store,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tapas Restaurant,Taxi Stand,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tourist Information Center,Toy / 

In [47]:
Street_grouped = Street_onehot.groupby('Neighborhood').mean().reset_index()
Street_grouped.head()

,Neighborhood,Yoga Studio,ATM,Accessories Store,Airport Lounge,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Stadium,Basketball Court,Bay,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bistro,Boarding House,Boat or Ferry,Bookstore,Border Crossing,Boutique,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buddhist Temple,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Campground,Canal,Candy Store,Cantonese Restaurant,Casino,Cha Chaan Teng,Cheese Shop,Chinese Breakfast Place,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Theater,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store,English Restaurant,Event Space,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hainan Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hong Kong Restaurant,Hookah Bar,Hospital,Hostel,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Indoor Play Area,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Laundromat,Leather Goods Store,Light Rail Station,Lighthouse,Lighting Store,Lingerie Store,Lounge,Luggage Store,Macanese Restaurant,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Mosque,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Night Market,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoor Sculpture,Outlet Store,Paella Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Peking Duck Restaurant,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Pier,Pizza Place,Playground,Plaza,Pool,Pop-Up Shop,Portuguese Restaurant,Pub,Public Art,Racetrack,Ramen Restaurant,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,River,Rock Climbing Spot,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Satay Restaurant,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Area,Smoke Shop,Snack Place,Soba Restaurant,Soccer Field,Soccer Stadium,Soup Place,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stables,Stadium,Stationery Store,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tapas Restaurant,Taxi Stand,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tourist Information Center,Toy / 

In [48]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [56]:
num_top_venues = 30

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Street_grouped['Neighborhood']

for ind in np.arange(Street_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Street_grouped.iloc[ind, :], num_top_venues)

In [57]:
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue
0,"AIRPORT BOULEVARD, sg",Airport Lounge,Boutique,Sandwich Place,Candy Store,Public Art,Ice Cream Shop,Food Court,Gift Shop,Border Crossing,Supermarket,Bubble Tea Shop,Shoe Store,Halal Restaurant,Garden,Burger Joint,Shopping Mall,Spa,Multiplex,Café,Snack Place,Fast Food Restaurant,Smoke Shop,Electronics Store,Hotel,BBQ Joint,Hobby Shop,Bakery,Waterfall,Coffee Shop,Cocktail Bar
1,"ALEXANDRA ROAD, sg",Chinese Restaurant,Bakery,Asian Restaurant,Soup Place,Bridge,Food Court,Thai Restaurant,Seafood Restaurant,Satay Restaurant,Dumpling Restaurant,Sporting Goods Shop,Juice Bar,Donut Shop,Fish & Chips Shop,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,Department Store,Dessert Shop,French Restaurant,Food Truck,Food Stand,Dim Sum Restaurant,Food & Drink Shop,Food,Flower Shop,Flea Market,Filipino Restaurant,Drugstore,Field
2,"ANG MO KIO AVENUE 3, sg",Food Court,Chinese Restaurant,Seafood Restaurant,Fast Food Restaurant,Noodle House,Supermarket,Indian Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop,General Entertainment,Soup Place,Bus Station,Field,Food Stand,Food Truck,Food & Drink Shop,Food,Flower Shop,Flea Market,Fried Chicken Joint,Fish & Chips Shop,Filipino Restaurant,French Restaurant,Women's Store,Farmers Market,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
3,"ANSON ROAD, sg",Japanese Restaurant,Coffee Shop,Café,Food Court,Hotel,Bakery,Fast Food Restaurant,Soup Place,Supermarket,Salad Place,Soba Restaurant,Sushi Restaurant,Indian Restaurant,Buffet,Restaurant,Chinese Restaurant,Gym / Fitness Center,Hotel Bar,Music Venue,Mediterranean Restaurant,Sandwich Place,Spa,Furniture / Home Store,Business Service,Shabu-Shabu Restaurant,Mosque,Shopping Mall,Bistro,Ramen Restaurant,Cocktail Bar
4,"BALESTIER ROAD, sg",Chinese Restaurant,Asian Restaurant,Indian Restaurant,Hotel,Seafood Restaurant,Dessert Shop,Food Court,Noodle House,Food,Fruit & Vegetable Store,Snack Place,Sculpture Garden,Grocery Store,Bakery,Café,Field,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Fast Food Restaurant,Food & Drink Shop,Flower Shop,Flea Market,Fish & Chips Shop,Filipino Restaurant,Food Stand,Food Truck,French Restaurant,Women's Store,Farmers Market,Fried Chicken Joint
